In [1]:
import os
import sys
import gensim
from collections import defaultdict
import pickle
import spacy

### This file create vectors for noun phrases in the vocabulary. It first loads vocabulary, documents and disctionary. Performs word2vec skipgram and then saves the vectors in a csv. Some vocabulary doesnt have vectors despite of min_count of 1 because in the previours steps documents with character length of less than 20 were filtered out. This can be changed and toyed with. 

In [2]:
#Load vocabulary

with open("/home/rin/CrossPlatform/Code/entity_vocab_onlyPROPN", 'r') as vfile:
    vocab = pickle.load(vfile)
    
#Load corpus
with open("/home/rin/CrossPlatform/Code/entity_corpus_onlyPROPN",'r') as cfile:
    corpus = pickle.load(cfile)
    
#Load dictionary that points phrases to underscore joined phrases
with open("/home/rin/CrossPlatform/Code/entity_dict_onlyPROPN", 'r') as dfile:
    vocab_dict = pickle.load(dfile)

In [3]:
#Text cleaner function

import re
from text_cleaner import remove, keep
from string import punctuation
from text_cleaner.processor.common import ASCII
from text_cleaner.processor.chinese import CHINESE, CHINESE_SYMBOLS_AND_PUNCTUATION
from text_cleaner.processor.misc import RESTRICT_URL
from nltk.corpus import stopwords

stopset = list(set(stopwords.words('english')))
with open("/home/rin/CrossPlatform/Code/stopwords.txt") as sfile:
    for line in sfile:
        stopset.append(line.strip().split()[0])
        
stopset = list(set(stopset))

def cleaning2(text):
    text = remove(text, [RESTRICT_URL]).lower()
    text = " ".join(text.split("."))
    text = " ".join(text.split(","))
    text = " ".join(text.split(":"))
    text = " ".join(text.split("!"))
    #text = re.sub(r'\b(?:(?:https?|ftp)://)?\w[\w-]*(?:\.[\w-]+)+\S*(?<![.,])', ' ', text.lower())
    #words = re.findall(r'[a-z.,]+', text.lower())
    clean_stop = [w for w in text.split(" ") if w not in stopset]
    return ' '.join(c for c in clean_stop if c not in punctuation)

In [4]:
#created stopwords removed, processed docs
processed_docs = []
for doc in corpus:
    processed_docs.append(cleaning2(doc))

In [5]:
processed_docs[28]

u'livestreaming national press club save persecuted christians launch event'

In [6]:
#create sentence pool
sentences = []

for p_d in processed_docs:
    sentences.append(p_d.split(" "))
    

In [7]:
%%time
from gensim.models import Word2Vec
model = Word2Vec(sentences, min_count=1)
print(model)

Word2Vec(vocab=33699, size=100, alpha=0.025)
CPU times: user 12 s, sys: 274 ms, total: 12.2 s
Wall time: 5.41 s


In [8]:
#write vectors to the file
not_in_voc=0
with open("/home/rin/CrossPlatform/Code/entity_vecs_onlyPROPN.csv", "w") as cfile:
    for v in vocab_dict.keys():
        try:
            vec = vocab_dict[v]
            row = vec + ","+  ",".join(map(str, model[vec]))
            cfile.write(row)
            cfile.write("\n")
        except:
            not_in_voc+=1
print not_in_voc       

9


/home/rin/.local/lib/python2.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [9]:
print len(vocab_dict.keys())

271
